In [1]:
# Load contractions model
from pycontractions import Contractions

cont = Contractions(api_key="glove-twitter-100")
cont.load_models()

### Parameters

In [2]:
file = 'Data/c1.csv'
ngram = (1,3)
mindf = 2
maxdf = 0.8

### Load

In [3]:
import pandas as pd

df = pd.read_csv(file,sep=",")
display(df.head())

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/f...,"Deer baiting, planned for Block Island by DEM,...",Richard August,08/26/2019,"animals,crime,environment,government regulatio...",politifact,TRUE
1,http://data.gesis.org/claimskg/creative_work/8...,Says 'Oregon is one of only three states that ...,Stop Gillnetting Now,05/30/2019,"environment,jobs,message machine 2012,recreation",politifact,FALSE
2,http://data.gesis.org/claimskg/creative_work/f...,NDP: “Canada went from 2.5 million protected l...,New Democratic Party,03/26/2019,"environment,ndp",factscan,OTHER
3,http://data.gesis.org/claimskg/creative_work/0...,Says Texas has so few gas-pump inspectors in t...,Hank Gilbert,03/25/2019,"consumer safety,energy,job accomplishments",politifact,FALSE
4,http://data.gesis.org/claimskg/creative_work/0...,Says Iowa Senate candidate Joni Ernst 'promise...,League of Conservation Voters,03/25/2019,"education,environment",politifact,TRUE


In [4]:
df5 = df.copy().sort_values(by="author")
df6 = df5.loc[(df5['organization'] != 'snopes')]
authors = df6["author"].unique()
#display(df6)

print(df6.shape)
len(authors)

(995, 7)


532

In [5]:
df1 = df.copy()
df1 = df1.loc[df1['rating'] != 'OTHER']
display(df1.head())

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/f...,"Deer baiting, planned for Block Island by DEM,...",Richard August,08/26/2019,"animals,crime,environment,government regulatio...",politifact,TRUE
1,http://data.gesis.org/claimskg/creative_work/8...,Says 'Oregon is one of only three states that ...,Stop Gillnetting Now,05/30/2019,"environment,jobs,message machine 2012,recreation",politifact,FALSE
3,http://data.gesis.org/claimskg/creative_work/0...,Says Texas has so few gas-pump inspectors in t...,Hank Gilbert,03/25/2019,"consumer safety,energy,job accomplishments",politifact,FALSE
4,http://data.gesis.org/claimskg/creative_work/0...,Says Iowa Senate candidate Joni Ernst 'promise...,League of Conservation Voters,03/25/2019,"education,environment",politifact,TRUE
5,http://data.gesis.org/claimskg/creative_work/0...,Says Donald Trump has outlined 'policies that ...,Michael Morell,03/25/2019,"candidate biography,economy,energy,foreign pol...",politifact,MIXTURE


In [6]:
t = df1.loc[(df1['rating'] == 'TRUE')]['id'].count()
f = df1.loc[(df1['rating'] == 'FALSE')]['id'].count()
m = df1.loc[(df1['rating'] == 'MIXTURE')]['id'].count()

print('True claims : %d' % t)
print('False claims : %d' % f)
print('Mixture claims : %d' % m)
print('TF claims : %d' % (t+f))

True claims : 146
False claims : 303
Mixture claims : 541
TF claims : 449


### Preprocess

In [7]:
import re

# def preprocess(sentence):
#     sentence = sentence.lower().strip()
#     translator = str.maketrans('’', "'", '')
#     sentence = sentence.translate(translator)
#     sentence = re.sub("u\.s\.","united states",sentence)
#     sentence = list(cont.expand_texts([sentence],precise=True))[0]
#     sentence = re.sub("[^a-zA-Z0-9_.’,]|(?<!\d)\.(?!\d)|(?<!\w)-(?!\w)|(?<!\d)\,(?!\d)",' ',sentence)
#     sentence = re.sub(",",'',sentence)
#     sentence = re.sub('\s+', ' ', sentence).strip()
#     sentence = re.sub(" s ",' ',sentence)
#     if sentence[0:5] == 'says ':
#         sentence = sentence[5:]
#     return sentence

def preprocess(sentence):
    sentence = sentence.lower().strip()
    translator = str.maketrans('’', "'", '')
    sentence = sentence.translate(translator)
    sentence = re.sub("u\.s\.","united states",sentence)
    sentence = list(cont.expand_texts([sentence],precise=True))[0]
    sentence = re.sub("[^a-zA-Z0-9_.’,]|(?<!\d)\.(?!\d)|(?<!\w)-(?!\w)|(?<!\d)\,(?!\d)",' ',sentence)
    sentence = re.sub(",",'',sentence)
    sentence = re.sub("\.",'',sentence)
    sentence = re.sub(" a ",' ',sentence)
    sentence = re.sub('\s+', ' ', sentence).strip()
    sentence = re.sub(" s ",' ',sentence)
    if sentence[0:5] == 'says ':
        sentence = sentence[5:]
    sentence = ' '.join([w for w in sentence.split() if len(w)>1])
    return sentence

In [8]:
for index in df1.index:
    df1.at[index,'text'] = preprocess(df1.at[index,'text'])
    rating = df1.at[index,'rating']
    if rating == 'FALSE':
        df1.at[index,'rating'] = 0
    if rating == 'TRUE':
        df1.at[index,'rating'] = 1
    if rating == 'MIXTURE':
        df1.at[index,'rating'] = 2

In [9]:
for index in df1.index:
    if index < 20:
        print("%s\n" % df1.at[index,'text'])

deer baiting planned for block island by dem is illegal in rhode island

oregon is one of only three states that allow the use of gillnets on inland waters

texas has so few gas pump inspectors in the field the hebs of this state the brookshires of this state they are calibrating their own gas pumps the texas department of agriculture just sends them the stickers and they calibrate them

iowa senate candidate joni ernst promises to shut down the department of education and abolish the epa

donald trump has outlined policies that read like kremlin wish list

there have been no cases of wolves killing people in rocky mountain states

the climate has not warmed in quite few years that is proven scientifically

the united states regulations are among the most difficult in the world

her campaign platform includes the biggest investment in new good paying jobs since world war ii

texas wildfires are linked to climate change

there is not one public study and by public study mean study avail

### Create additional dataframes

In [10]:
#df1 = true/false/mixed
#df2 = true/false
#df3 = true-false/mixed

In [11]:
df2 = df1.copy()
df2 = df2.loc[(df2['rating'] == 0) | (df2['rating'] == 1)]
df2.head()

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/f...,deer baiting planned for block island by dem i...,Richard August,08/26/2019,"animals,crime,environment,government regulatio...",politifact,1
1,http://data.gesis.org/claimskg/creative_work/8...,oregon is one of only three states that allow ...,Stop Gillnetting Now,05/30/2019,"environment,jobs,message machine 2012,recreation",politifact,0
3,http://data.gesis.org/claimskg/creative_work/0...,texas has so few gas pump inspectors in the fi...,Hank Gilbert,03/25/2019,"consumer safety,energy,job accomplishments",politifact,0
4,http://data.gesis.org/claimskg/creative_work/0...,iowa senate candidate joni ernst promises to s...,League of Conservation Voters,03/25/2019,"education,environment",politifact,1
6,http://data.gesis.org/claimskg/creative_work/0...,there have been no cases of wolves killing peo...,Oregon Department of Fish and Wildlife,03/25/2019,environment,politifact,1


In [12]:
df3 = df1.copy()
for index in df3.index:
    rating = df3.at[index,'rating']
    if rating == 1:
        df3.at[index,'rating'] = 0
    if rating == 2:
        df3.at[index,'rating'] = 1
        
df3.head()

,id,text,author,date,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/f...,deer baiting planned for block island by dem i...,Richard August,08/26/2019,"animals,crime,environment,government regulatio...",politifact,0
1,http://data.gesis.org/claimskg/creative_work/8...,oregon is one of only three states that allow ...,Stop Gillnetting Now,05/30/2019,"environment,jobs,message machine 2012,recreation",politifact,0
3,http://data.gesis.org/claimskg/creative_work/0...,texas has so few gas pump inspectors in the fi...,Hank Gilbert,03/25/2019,"consumer safety,energy,job accomplishments",politifact,0
4,http://data.gesis.org/claimskg/creative_work/0...,iowa senate candidate joni ernst promises to s...,League of Conservation Voters,03/25/2019,"education,environment",politifact,0
5,http://data.gesis.org/claimskg/creative_work/0...,donald trump has outlined policies that read l...,Michael Morell,03/25/2019,"candidate biography,economy,energy,foreign pol...",politifact,1


In [13]:
dfs = dict()
dfs['3 Classes'] = df1
dfs['2 Classes'] = df2
dfs['2 Mixed Classes'] = df3

### Vectors

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize(df):
    vectorizer = TfidfVectorizer(ngram_range=ngram,min_df=mindf,max_df=maxdf)
    vectors = vectorizer.fit_transform(df['text']).toarray()
    #print(vectorizer.get_feature_names())
    return vectors

### Classification

In [15]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

def classify(df, key):
    X = vectorize(df)
    y = df["rating"]
    y = y.astype('int')
    clf = LinearSVC(random_state=0, tol=1e-5)
    k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    score = cross_val_score(clf, X, y, cv=k_fold, scoring='accuracy')
    print('%s - Mean accuracy: %f, Deviation: %f' % (key,score.mean(),score.std()))

In [16]:
for key in dfs:
    classify(dfs[key], key)

3 Classes - Mean accuracy: 0.524244, Deviation: 0.030141
2 Classes - Mean accuracy: 0.670307, Deviation: 0.031081
2 Mixed Classes - Mean accuracy: 0.546638, Deviation: 0.049576
